# LangChain API Tutorial

This tutorial demonstrates how to use LangChain's core functionalities.
LangChain is a powerful framework designed to facilitate building language model-powered applications.
We'll explore its components, including prompt creation, chains, retrieval, and agents.

In [3]:
# Use this to enable vim shortcuts.
# !sudo /bin/bash -c "(source /venv/bin/activate; pip install --quiet jupyterlab-vim)"
# !jupyter labextension enable

In [2]:
%load_ext autoreload
%autoreload 2

## Setting Up

We'll start by setting up the environment and initializing our language model (`ChatOpenAI`).
Here, we're using OpenAI's `gpt-4o-mini` model with a temperature of 0 for deterministic outputs.

In [1]:
#!sudo /venv/bin/pip install langchain --quiet
#!sudo /venv/bin/pip install -U langchain-community --quiet
#!sudo /venv/bin/pip install -U langchain-openai --quiet
#!sudo /venv/bin/pip install -U langchain-core --quiet
#!sudo /venv/bin/pip install -U langchainhub --quiet
#!sudo /venv/bin/pip install -U unstructured --quiet
#!sudo /venv/bin/pip install --quiet chromadb

In [4]:
import logging
import os
import random
import time

import helpers.hdbg as hdbg
import langchain
import langchain.agents as lngchagents
import langchain.document_loaders.csv_loader as csvloader
import langchain.hub
import langchain.prompts as lngchprmt
import langchain.schema.messages as lnchscme
import langchain.schema.runnable as lngchschrun
import langchain_community.vectorstores as vectorstores
import langchain_core.output_parsers as lngchoutpar
import langchain_openai as lngchopai

In [23]:
# Avoid messages from OpenAI REST interface.
hdbg.init_logger(verbosity=logging.CRITICAL)

_LOG = logging.getLogger(__name__)

## Add your OpenAPI key and initiate GPT model

In [8]:
# Add OpenAPI to environment variable.
#os.environ["OPENAI_API_KEY"] = ""
# Initiate OpenAI model.
chat_model = lngchopai.ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Message Handling with `HumanMessage` and `SystemMessage`

LangChain uses `HumanMessage` and `SystemMessage` objects to structure conversations.
- **`SystemMessage`**: Defines the behavior of the assistant.
- **`HumanMessage`**: Represents user input.

Let's see how this works in practice with some example messages.

In [26]:
# Define system behavior and user input.
messages = [
    lnchscme.SystemMessage(
        content="You're an assistant knowledgeable about healthcare."
    ),
    lnchscme.HumanMessage(content="What is Medicaid managed care?"),
]
# Generate a response.
val = chat_model.invoke(messages)

In [27]:
type(val)

langchain_core.messages.ai.AIMessage

In [28]:
print(val.content)

Medicaid managed care is a system of delivering Medicaid services through private health insurance plans. In this model, state Medicaid programs contract with private insurance companies to provide a range of healthcare services to Medicaid beneficiaries. Here are some key features of Medicaid managed care:

1. **Care Coordination**: Managed care organizations (MCOs) are responsible for coordinating care for their members, which can lead to more integrated and efficient healthcare services.

2. **Cost Control**: By using a managed care model, states aim to control costs while ensuring that beneficiaries receive necessary medical services. MCOs are incentivized to provide preventive care and manage chronic conditions to reduce overall healthcare spending.

3. **Provider Networks**: MCOs typically establish networks of healthcare providers, including doctors, hospitals, and specialists. Beneficiaries usually need to receive care from providers within the network to have their services co

In [14]:
val.usage_metadata

{'input_tokens': 24,
 'output_tokens': 348,
 'total_tokens': 372,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

## Restricting Assistant's Scope

You can further control the assistant's responses by tailoring the `SystemMessage`.
For instance, we'll restrict it to only answer healthcare-related questions.

In [15]:
messages = [
    lnchscme.SystemMessage(
        content="You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."
    ),
    lnchscme.HumanMessage(content="How do I change a tire?"),
]
val = chat_model.invoke(messages)
print(val.content)

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I'm here to help with healthcare-related questions. If you have any health concerns or questions about medical topics, feel free to ask!


## Creating Custom Prompts with `ChatPromptTemplate`

Custom prompts are essential for tasks like summarization, question answering, or content generation.
We'll use `ChatPromptTemplate` to define structured prompts and format them dynamically.

In [16]:
# Define a custom template for hospital reviews.
review_template_str = """
Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up
any information that's not from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = lngchprmt.ChatPromptTemplate.from_template(review_template_str)

# Provide context and a question.
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

# Format the template.
print(review_template.format(context=context, question=question))

Human: 
Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up
any information that's not from the context. If you don't know an answer, say you don't know.

I had a great stay!

Did anyone have a positive experience?



## Advanced Prompt Structuring

Sometimes, you may need to structure prompts with multiple components, like system instructions and user input.
`SystemMessagePromptTemplate` and `HumanMessagePromptTemplate` allow fine-grained control.

In [17]:
# Define system and human message templates.
review_system_prompt = lngchprmt.SystemMessagePromptTemplate(
    prompt=lngchprmt.PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

review_human_prompt = lngchprmt.HumanMessagePromptTemplate(
    prompt=lngchprmt.PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

# Combine into a chat prompt template.
review_prompt_template = lngchprmt.ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=[review_system_prompt, review_human_prompt],
)

# Format messages.
formatted_messages = review_prompt_template.format_messages(
    context=context, question=question
)
print(formatted_messages)

[SystemMessage(content="\nYour job is to use patient reviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed as possible, but don't make up\nany information that's not from the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDid anyone have a positive experience?\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='Did anyone have a positive experience?', additional_kwargs={}, response_metadata={})]


In [21]:
print(formatted_messages[0].content)


Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up
any information that's not from the context. If you don't know an answer, say you don't know.

I had a great stay!

Did anyone have a positive experience?



In [22]:
print(formatted_messages[1].content)

Did anyone have a positive experience?


## Chains

Chains are a fundamental abstraction in LangChain, allowing you to combine prompts and models into workflows.
We'll create a simple chain to answer questions based on a given context.

In [29]:
# Create a chain using the template and chat model.
output_parser = lngchoutpar.StrOutputParser()
review_chain = review_prompt_template | chat_model | output_parser

# Test the chain.
review_chain.invoke({"context": context, "question": question})

'Yes, one patient mentioned that they had a great stay at the hospital, indicating a positive experience.'

## Retrieval with FAISS

FAISS (Facebook AI Similarity Search) is used to efficiently retrieve relevant documents based on similarity scores.
We'll demonstrate how to load a dataset, create embeddings, and retrieve documents.

In [39]:
import pandas as pd

REVIEWS_CSV_PATH = "data/reviews.csv"

df = pd.read_csv(REVIEWS_CSV_PATH)
df.head(3)

,review_id,visit_id,review,physician_name,hospital_name,patient_name
0,0,6997,The medical staff at the hospital were incredi...,Laura Brown,Wallace-Hamilton,Christy Johnson
1,9,8138,The hospital's commitment to patient education...,Steven Watson,Wallace-Hamilton,Anna Frazier
2,11,680,The hospital's commitment to patient safety wa...,Chase Mcpherson Jr.,Wallace-Hamilton,Abigail Mitchell


In [37]:
REVIEWS_CHROMA_PATH = "chroma_data"

# Load reviews dataset.
loader = csvloader.CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")
reviews = loader.load()

# Create a vector store.
reviews_vector_db = vectorstores.Chroma.from_documents(
    reviews, lngchopai.OpenAIEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH
)

In [42]:
# Retrieve relevant documents.
question = "Has anyone complained about communication with the hospital staff?"
relevant_docs = reviews_vector_db.similarity_search(question, k=3)

print("#0\n" + relevant_docs[0].page_content)
print("#1\n" + relevant_docs[1].page_content)
print("#2\n" + relevant_docs[2].page_content)

#0
review_id: 73
visit_id: 7696
review: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.
physician_name: Maria Thompson
hospital_name: Little-Spencer
patient_name: Terri Smith
#1
review_id: 73
visit_id: 7696
review: I had a frustrating experience at the hospital. The communication between the medical staff and me was unclear, leading to misunderstandings about my treatment plan. Improvement is needed in this area.
physician_name: Maria Thompson
hospital_name: Little-Spencer
patient_name: Terri Smith
#2
review_id: 785
visit_id: 2593
review: My stay at the hospital was challenging. The medical care was adequate, but the lack of communication from the staff created some frustration.
physician_name: Brittany Harris
hospital_name: Jones, Taylor and Garcia
patient_name: Ryan Jacobs


## Building a Retrieval-Enhanced QA Chain

We'll combine retrieval and prompts to build a more dynamic question-answering system.
This chain retrieves relevant documents and uses them as context for the assistant.

In [43]:
# Create a retriever.
reviews_retriever = reviews_vector_db.as_retriever(k=10)

# Build the QA chain.
review_chain = (
    {"context": reviews_retriever, "question": lngchschrun.RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | lngchoutpar.StrOutputParser()
)

# Test the QA chain.
question = "Has anyone complained about communication with the hospital staff?"
review_chain.invoke(question)

'Yes, there are complaints about communication with the hospital staff. One patient, Terri Smith, expressed frustration due to unclear communication between the medical staff and herself, which led to misunderstandings about her treatment plan at Little-Spencer hospital. Another patient, Ryan Jacobs, also mentioned that the lack of communication from the staff created frustration during his stay at Jones, Taylor and Garcia hospital, despite the medical care being adequate.'

## Agents

Agents are more flexible than chains, as they can decide the sequence of actions to take.
We'll demonstrate an agent that can answer questions using tools for reviews and wait times.

- The chain is hardwired
- An agent is an LLM that decides the sequence of actions to execute.

In [44]:
def get_current_wait_time(hospital: str) -> int | str:
    """
    Dummy function to generate fake wait times.
    """
    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"
    # Simulate API call delay.
    time.sleep(1)
    return random.randint(0, 10000)

In [45]:
# Tool is an interface that an agent uses to interact with a function.
# Each description explains the Agent when to call each tool.
tools = [
    lngchagents.Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    lngchagents.Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

hospital_agent_prompt = langchain.hub.pull("hwchase17/openai-functions-agent")

hospital_agent = lngchagents.create_openai_functions_agent(
    llm=chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

# Agent run-time.
hospital_agent_executor = lngchagents.AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

/venv/lib/python3.12/site-packages/langsmith/client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [46]:
hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital C?"}
)



> Entering new AgentExecutor chain...

Invoking: `Waits` with `C`


9223The current wait time at hospital C is 9223 minutes. This seems unusually high; please double-check the hospital name or consider contacting the hospital directly for accurate information.

> Finished chain.


{'input': 'What is the current wait time at hospital C?',
 'output': 'The current wait time at hospital C is 9223 minutes. This seems unusually high; please double-check the hospital name or consider contacting the hospital directly for accurate information.',
 'intermediate_steps': [(AgentActionMessageLog(tool='Waits', tool_input='C', log='\nInvoking: `Waits` with `C`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"C"}', 'name': 'Waits'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_86d0290411'}, id='run-a697b9be-fb12-46d5-bd73-eb76d7daf6ac')]),
   9223)]}

In [47]:
hospital_agent_executor.invoke(
    {"input": "What have patients said about their comfort at the hospital?"}
)



> Entering new AgentExecutor chain...

Invoking: `Reviews` with `What have patients said about their comfort at the hospital?`


Patients have expressed mixed feelings about their comfort at the hospital. One patient, Jasmine Patel DDS, praised Brown-Golden hospital for its dedication to patient comfort, highlighting the well-designed private rooms and comfortable furnishings that made her recovery more bearable and contributed to an overall positive experience.

In contrast, another patient, Tammy Benson DVM, shared her experience at Garcia Ltd hospital, where she appreciated the staff's dedication to patient care but noted that the uncomfortable beds made it difficult for her to get a good night's sleep during her stay. 

Overall, while some patients found the environment conducive to comfort, others faced challenges with the sleeping arrangements.Patients have shared mixed experiences regarding their comfort at the hospital. For instance, Jasmine Patel DDS praised Brown-Golden Hos

{'input': 'What have patients said about their comfort at the hospital?',
 'output': "Patients have shared mixed experiences regarding their comfort at the hospital. For instance, Jasmine Patel DDS praised Brown-Golden Hospital for its focus on patient comfort, noting the well-designed private rooms and comfortable furnishings that enhanced her recovery experience.\n\nOn the other hand, Tammy Benson DVM expressed her appreciation for the staff's dedication to patient care at Garcia Ltd Hospital but mentioned that the uncomfortable beds made it hard for her to get a good night's sleep during her stay.\n\nIn summary, while some patients found the environment supportive of their comfort, others encountered issues with sleeping arrangements.",
 'intermediate_steps': [(AgentActionMessageLog(tool='Reviews', tool_input='What have patients said about their comfort at the hospital?', log='\nInvoking: `Reviews` with `What have patients said about their comfort at the hospital?`\n\n\n', message_l